MODELE PREDICTIVE DE L'INDICE DE LA CAPACITÉ DE GESTION DES ENTREPRISES DANS LES PAYS EN DÉVELOPPEMENT

Installation des librairies

In [ ]:
## Importation des Bibliothéques requises
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import pickle 
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression



Importation des données d'enquête sur les entreprises informelles des pays en développement

In [ ]:
# Importation des données Chemin vers votre fichier .dta
df = pd.read_stata("C:/Users/User/Documents/Zoom/Candidature 2025/Formation Sorbonne Data Analytics/Introduction to Python/Projet python/assets/Informal-Sector-Enterprise-Surveys-Indicators-Database_February_3_2025.dta")
# Description des données
df.describe()
df.info()
# Encodage de la variable 'Pays'
label_encoder=LabelEncoder()
df['Country_Encoded'] = label_encoder.fit_transform(df['country'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14957 entries, 0 to 14956
Data columns (total 99 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idstd        14957 non-null  float64
 1   country      14957 non-null  object 
 2   country_abr  14957 non-null  object 
 3   cityx        14957 non-null  object 
 4   sec1         14956 non-null  float32
 5   sec2         14956 non-null  float32
 6   sec3         14956 non-null  float32
 7   dem1         14957 non-null  float32
 8   dem2         14957 non-null  float32
 9   dem3         14861 non-null  float32
 10  dem4         14891 non-null  float32
 11  dem5         14957 non-null  float32
 12  dem6         14957 non-null  float32
 13  dem7         14957 non-null  float32
 14  dem8         14636 non-null  float32
 15  dem9         14650 non-null  float32
 16  dem10        13578 non-null  float32
 17  dem11        13893 non-null  float32
 18  dem12        14899 non-null  float32
 19  dem1

In [ ]:
df.describe()
import pickle as pkl

Simulation de l'entraînement du modèle

In [ ]:
def train_management_index_prediction_model():

    X= df[['Country_Encoded','dem2','dem6','dem7','wf1','ge1','op1','op9','tech2','tech3']]
    y = df['mg1'] 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model



Sauvegarde du modèle simulé

In [ ]:
# Sauvegarder le modèle simulé
model = train_management_index_prediction_model()
pickle.dump(model, open('modèle de prédiction de la capacité de gestion des entreprises.pkl', 'wb'))
# --- 2. Fonction de prédiction ---
def predict_workers(Country_Encoded,dem2,dem6,dem7,wf1,ge1,op1,op9,tech2,tech3):
    input_data = np.array([[Country_Encoded,dem2,dem6,dem7,wf1,ge1,op1,op9,tech2,tech3]])
    prediction = model.predict(input_data)
    return max(0, int(round(prediction[0]))) # S'assurer que la prédiction n'est pas négative

Interface Streamlit

In [ ]:
# --- 3. Interface Streamlit ---
st.title("Prédiction de l'indice de la capacité de gestion de l'entreprise")

st.subheader("Veuillez entrer les caractéristiques de l'entreprise :")
# Liste déroulante pour le pays
pays_options = df['country'].unique()
pays_choisi = st.selectbox("Pays d'appartenance de l'entreprise", pays_options)
country_encode = label_encoder.transform([pays_choisi])[0]
# Input pour le niveau d'étude secondaire
dem2 = st.slider("Niveau d'étude secondaire du propriétaire principal (0=Non, 100=Oui)", 0, 100, 100, step=100)
# Input pour l'âge de l'entreprise
dem6 = st.number_input("Âge de l'entreprise", min_value=0, step=1, value=5)
# Input pour l'âge moyen des propriétaires
dem7 = st.number_input("Âge du propriétaire ou âge moyen des propriétaires", min_value=0, step=1, value=40)
# Input pour le nombre moyen d'employés
wf1 = st.number_input("Nombre moyen d'employés", min_value=0, step=1, value=2)
# Input pour le sexe féminin des propriétaires
ge1 = st.slider("Le sexe du proprétaire ou des propriétaires principales est féminin (0=Non, 100=Oui)", 0, 100, 0, step=100)
# Input pour l'enregistrement de profit
op1 = st.slider("Les entreprises enregistrant un profit le mois passé (0=Non, 100=Oui)", 0, 100, 100, step=100)
# Input pour le nombre moyen d'heures d'ouverture
op9= st.number_input("Nombre moyen d'heures d'ouverture dans la semaine", min_value=0, step=1, value=40)
# Input pour l'utilisation du mobile money
tech2= st.slider("Les entreprises utilisant le mobile money (0=Non, 100=Oui)", 0, 100, 100, step=100)
# Input pour l'utilisation d'un ordinateur/tablette
tech3= st.slider("Les entreprises utilisant un ordinateur/tablette (0=Non, 100=Oui)", 0, 100, 0, step=100)
# Bouton de prédiction
if st.button("Prédire l'Indice de Gestion"):
    input_data = pd.DataFrame({
        'Pays_Encoded': [pays_encode],
        'dem2': [secondaire],
        'dem6': [age_entreprise],
        'dem7': [age_proprietaire],
        'wf1': [nb_employes],
        'ge1': [sexe_feminin],
        'op1': [profit],
        'op9': [heures_ouverture],
        'tech2': [mobile_money],
        'tech3': [ordinateur]
    })
    prediction = model.predict(input_data)
    st.subheader(f"L'indice de capacité de gestion prédit est : {prediction[0]:.2f}")

st.caption("Ceci est un modèle de prédiction basé sur les données fournies.")

#dem2_input = st.number_input("Le propriétaire principal a terminé les études sécondaires", min_value=0.0, step=1.0)
#dem6_input = st.number_input("Âge moyen de l'entreprise (en années)", min_value=0.0, step=1.0)
#dem7_input = st.number_input("Âge moyen du propriétaire (en années)", min_value=0.0, step=1.0)
#wf1_input = st.number_input("Nombre moyen d'employés de l'entreprise (en années)", min_value=0.0, step=1.0)
#ge1_input = st.number_input("Les propriétaires principales sont des femmes", min_value=0.0, step=1.0)
#op1_input = st.number_input("Les entreprises enregistrant un profit le mois passé", min_value=0.0, step=1.0)
#op9_input = st.number_input("Nombre moyen d'heures d'ouverture de la semaine ", min_value=0.0, step=1.0)
#tech2_input = st.number_input("Les entreprises utilisant le mobile money", min_value=0.0, step=1.0)
#tech3_input = st.number_input("Les entreprises utilisant un ordinataire/tablette", min_value=0.0, step=1.0)
# Charger le modèle pré-entraîné
#loaded_model = pickle.load(open('modèle de prédiction de la capacité de gestion.pkl','rb'))

#if st.button("Prédire l'index de capacité ge gestion"):
    #prediction = predict_workers(loaded_model, demo6_input, demo7_input, op9_input)
    #st.subheader(f"L'indice de capacité de gestion est : {prediction}")

#st.sidebar.header("À propos de l'application")
#st.sidebar.info("Cette application prédit le nombre de travailleurs par mois en fonction de l'âge moyen de l'entreprise, de l'âge moyen du propriétaire et du nombre d'heures d'ouverture par semaine.")

2025-05-16 07:47:04.934 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-05-16 07:47:27.990 
  command:

    streamlit run c:\Users\User\Documents\Zoom\Candidature 2025\Formation Sorbonne Data Analytics\Introduction to Python\Projet python\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-16 07:47:27.993 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 07:47:27.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 07:47:27.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 07:47:28.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 07:47:28.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 07:47:28.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 07:

DeltaGenerator()